In [190]:
# -*- coding: utf-8 -*
import os
import re
import shutil
import textract
import numpy as np
import pandas as pd
# NLTK
from nltk import tokenize
from nltk import SnowballStemmer
from nltk.corpus import stopwords
# Import punctuation marks
from string import punctuation
from tqdm.notebook import tqdm
from PyPDF2 import PdfFileReader

## São Bento Abade

In [191]:
root_dir = "../data/381-licitacoes-sao-bento-abade/data/files/"
file_description = pd.read_json(os.path.join(root_dir,"file_description.jsonl"), lines=True)
file_description = file_description.loc[file_description['type'] == 'pdf']
file_description

,url,file_name,crawler_id,instance_id,crawled_at_date,referer,type,extracted_files
0,https://www.saobentoabade.mg.gov.br/site/index...,45af5cc070110fb2b6dc1782daeb1175.pdf,7,160772537966722,2020-12-11 22:27:31.975482,https://www.saobentoabade.mg.gov.br/site/index...,pdf,[/home/lucas/Sandbox/coletasbranchnova/saobent...
1,https://www.saobentoabade.mg.gov.br/site/image...,1f3341125c619eec536ab1ffa47e5325.pdf,7,160772537966722,2020-12-11 22:28:07.338505,https://www.saobentoabade.mg.gov.br/site/index...,pdf,[]
2,https://www.saobentoabade.mg.gov.br/site/image...,3cbbe4f564c977b46dde47dabb5b7f0c.pdf,7,160772537966722,2020-12-11 22:28:10.514126,https://www.saobentoabade.mg.gov.br/site/index...,pdf,[]
3,https://www.saobentoabade.mg.gov.br/site/index...,8b6725a485eb0fb641cf7b905163d347.pdf,7,160772537966722,2020-12-11 22:28:14.146812,https://www.saobentoabade.mg.gov.br/site/index...,pdf,[/home/lucas/Sandbox/coletasbranchnova/saobent...
4,https://www.saobentoabade.mg.gov.br/site/index...,1cf082dda6251afbebce467c8e0f34bd.pdf,7,160772537966722,2020-12-11 22:28:21.965386,https://www.saobentoabade.mg.gov.br/site/index...,pdf,[/home/lucas/Sandbox/coletasbranchnova/saobent...
...,...,...,...,...,...,...,...,...
268,https://www.saobentoabade.mg.gov.br/site/image...,0349675a55b77595aeb7020e12ee9958.pdf,7,160772537966722,2020-12-11 22:54:03.239815,https://www.saobentoabade.mg.gov.br/site/index...,pdf,[/home/lucas/Sandbox/coletasbranchnova/saobent...
269,https://www.saobentoabade.mg.gov.br/site/index...,b8d5a9c794d0c10ab2a4030b797b1f2e.pdf,7,160772537966722,2020-12-11 22:54:06.352912,https://www.saobentoabade.mg.gov.br/site/index...,pdf,[/home/lucas/Sandbox/coletasbranchnova/saobent...
270,https://www.saobentoabade.mg.gov.br/site/image...,91700bfc0d3e9cab7f6febbb93730bd1.pdf,7,160772537966722,2020-12-11 22:54:21.100184,https://www.saobentoabade.mg.gov.br/site/index...,pdf,[]
272,https://www.saobentoabade.mg.gov.br/site/index...,e9f1d49b2cd9dd197f508e02e1c4fc4c.pdf,7,160772537966722,2020-12-11 22:54:25.039750,https://www.saobentoabade.mg.gov.br/site/index...,pdf,[/home/lucas/Sandbox/coletasbranchnova/saobent...


In [192]:
file_description['referer'].unique()

array(['https://www.saobentoabade.mg.gov.br/site/index.php/editais?limit=0'],
      dtype=object)

In [193]:
file_description.shape

(232, 8)

In [194]:
# Test Textract
text = str(textract.process(os.path.join(root_dir,
                                         '45af5cc070110fb2b6dc1782daeb1175.pdf'),
                            encoding='utf-8'), 'utf-8')
rom_stemmer = SnowballStemmer('portuguese')
portuguese_stw = stopwords.words('portuguese') + list(punctuation)
# Remove stopwords and punctuation
# Character class to match punctuation.  The dash ('-') is special
# in character classes, so put a backslash in front of it to make
# it just a literal dash.
_char_class_punct = "[" + re.escape(punctuation) + "]"

# Pattern: a punctuation character followed by one or more punctuation characters.
# Thus, a run of two or more punctuation characters.
_pat_punct_run = re.compile(_char_class_punct + _char_class_punct + '+')
no_stop_text = filter(lambda x: x not in portuguese_stw,
                      tokenize.word_tokenize(text.lower(), language='portuguese'))
# Remove repeated punctuation
no_stop_text = filter(lambda x: False if re.match(_pat_punct_run, x) else True, no_stop_text)
# Reduce words to stem
stemmed_text = list(map(rom_stemmer.stem, no_stop_text))
print(stemmed_text)

['prefeitur', 'municipal', 'bent', 'abade-mg', 'administr', '2017-2020', 'edital', 'licit', 'process', 'nº', '052/2017', 'modal', 'pregã', 'presencial', 'nº', '033/2017', 'reg', 'lei', 'nº', '10.520', '17', 'julh', '2002', 'subsidiari', 'lei', 'nº', '8.666', '21/06/93', 'alter', 'lei', 'nº8883/94', 'lei', 'nº', '9.648/98', 'decret', '3.555/2000', 'port', 'n.º', '005/2017', 'nomeaçã', 'pregoeir', '05', 'janeir', '2017', 'tip', 'menor', 'prec', 'unitári', 'objet', 'registr', 'prec', 'futur', 'possível', 'aquisiçã', 'pneus', 'nov', 'sel', 'inmetr', 'manutençã', 'frot', 'municípi', 'bent', 'abad', 'mg', 'conform', 'descriçã', 'constant', 'term', 'referent', '–', 'anex', 'i', 'edital', 'receb', 'envelop', 'document', 'propost', 'prec', 'dia', '17/10/2017', 'hor', '10h', 'abertur', 'local', 'pac', 'municipal', 'sal', 'licit', 'interess', 'dev', 'entreg', 'document', 'credenc', 'declar', 'cumpriment', 'requisit', 'habilit', 'declar', 'inexistent', 'fat', 'impedit', 'cartã', 'cnpj', 'serv', 'c

In [195]:
text

'PREFEITURA MUNICIPAL DE SÃO BENTO ABADE-MG\nADMINISTRAÇÃO 2017-2020\n\nEDITAL DE LICITAÇÃO\nPROCESSO Nº 052/2017\nMODALIDADE: PREGÃO PRESENCIAL Nº 033/2017\n(Regida pela Lei nº 10.520, de 17 de julho de 2002 e subsidiariamente pela Lei nº 8.666 de\n21/06/93 (com as alterações da Lei nº8883/94 e da Lei nº 9.648/98) e Decreto 3.555/2000.\nPortaria n.º 005/2017, nomeação do Pregoeiro, de 05 de janeiro de 2017\nTipo: Menor Preço unitário\nObjeto: Registro de preços para futura e possível aquisição de pneus novos, com selo do\nINMETRO para manutenção da frota do Município de São Bento Abade - MG, conforme\ndescrição, constantes no termo de referência – ANEXO I do edital.\nDO RECEBIMENTO DOS ENVELOPES DE DOCUMENTAÇÃO E PROPOSTA DE PREÇOS\nDia: 17/10/2017\nHora: 10h - Abertura\nLocal: Paço Municipal (Sala de Licitações)\nOs interessados deverão entregar o Documento de Credenciamento, a Declaração de\nCumprimento dos Requisitos de Habilitação, Declaração de inexistência de fatos impeditivos e

In [196]:
def identifica_atas_df(file_description):
    # Create flags for word ata
    file_description['ata_from_url'] = 0
    file_description['ata_from_file_name'] = 0
    file_description['ata_from_file_title'] = 0
    file_description['ata_from_file_content'] = 0
    # Check urls that contains the word ata
    ata_regex = 'ata|Ata|aTa|atA|ATA'
    file_description.loc[file_description.url.str.split("/").str[-1].str.contains(ata_regex, regex=True), 'ata_from_file_name'] = 1
    file_description.loc[file_description.url.str.contains(ata_regex, regex=True), 'ata_from_url'] = 1
    # Check file content
    for idx, row in tqdm(file_description.iterrows()):
        try:
            pdf = PdfFileReader(os.path.join(root_dir, row['file_name']))
        except Exception as e:
            print("{0}: {1}".format(row['file_name'],e))
        if pdf is None:
            print(row['file_name'])
            pass
        try:
            pdf_num_pages = pdf.getNumPages()
            pdf_doc_info = pdf.getDocumentInfo()
        except:
            pass
        # Procurar ocorrencias da string "ata" nos títulos
        try:
            # Procurar a string nos títulos como lowercase, para evitar problemas de matching
            if 'ata' in pdf_doc_info['/Title'].lower():
                file_description.loc[idx, 'ata_from_file_title'] = 1
        except:
            pass
        # read only the first 5 pages
        pages_to_read = pdf_num_pages if pdf_num_pages <= 5 else 5
        for i in range(pages_to_read):
            try:
                page = pdf.getPage(i).extractText().lower().replace('\n', '')
            except Exception:
                pass
            if re.search(r'\bata\b', page):
                file_description.loc[idx, 'ata_from_file_content'] = 1
                break
    return file_description

In [197]:
file_description = identifica_atas_df(file_description)
# Usando o .sum() aqui pois como os valores da coluna são 0 ou 1,
# achei que ficava mais claro para ver em quantos docs a gente
# pode identificar uma ata. O .size() tava meio confuso pra mim.
file_description[[
    'ata_from_url',
    'ata_from_file_name',
    'ata_from_file_title',
    'ata_from_file_content'
]].sum()

ata_from_url              62
ata_from_file_name        47
ata_from_file_title       37
ata_from_file_content    158
dtype: int64

#### Investigar arquivos que contém a string 'ata' na URL

In [198]:
file_description[file_description['ata_from_url'] == 1]['url'].values

array(['https://www.saobentoabade.mg.gov.br/site/images/phocadownload/2020/pl-027-2020-ata-propostas.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/atas/ADENDO_I_ao_Edital_Processo_080_-_Preg%C3%A3o_039.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/atas/Contrato_002_-_processo_012_-_inexigibilidade_002_2020.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/atas/Ata_da_sessao_-_Processo_041_-_Pregao_015_-_sulfite.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/atas/Ata_de_Registro_-_37_-_GC_Lab.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/2020/pl-027-2020-ata-habilitacao-e-propostas.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/2020/pl-044-2020_-_Ata_habilitacao.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/atas/02_ATA_HOMOLOGA%C3%87%C3%83O_PROCESSO_AP%C3%93S_LIBERA%C3%87%C3%83O_TCE_MG.

##### Uma verificação visual dos arquivos e das URLs mostra que atas foram corretamente identificadas, mas alguns outros documentos (falsos positivos) estão presentes, exemplos: 
1. https://www.saobentoabade.mg.gov.br/site/images/phocadownload/atas/ADENDO_I_ao_Edital_Processo_080_-_Preg%C3%A3o_039.pdf
2. https://www.saobentoabade.mg.gov.br/site/images/phocadownload/atas/Contrato_002_-_processo_012_-_inexigibilidade_002_2020.pdf
3. https://www.saobentoabade.mg.gov.br/site/images/phocadownload/atas/Contrato_001_2020_-_Processo_011_-_Inexigibilidade_001.pdf

#### Investigar arquivos que contém a string 'ata' no nome do arquivo

In [199]:
file_description[file_description['ata_from_file_name'] == 1]['url'].values

array(['https://www.saobentoabade.mg.gov.br/site/images/phocadownload/2020/pl-027-2020-ata-propostas.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/atas/Ata_da_sessao_-_Processo_041_-_Pregao_015_-_sulfite.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/atas/Ata_de_Registro_-_37_-_GC_Lab.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/2020/pl-027-2020-ata-habilitacao-e-propostas.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/2020/pl-044-2020_-_Ata_habilitacao.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/atas/02_ATA_HOMOLOGA%C3%87%C3%83O_PROCESSO_AP%C3%93S_LIBERA%C3%87%C3%83O_TCE_MG.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/atas/Ata_de_Registro_-_36_-_Biotcnica.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/atas/01_ATA_SUSPENS%C3%83O_PROCESSO_ENVIADO_AO_TCE_MG.pdf',
       'https://w

Todos os arquivos que foram identificados com a string 'ata' no nome do arquivo, também foram identificados com a mesma string na URL. Porém alguns arquivos possuem a string na URL mas não no nome. Estes podem ser falsos positivos.

In [200]:
# Testar arquivo com a string 'ata' na URL, mas não no nome
file_description[(file_description['ata_from_file_name'] == 0) &
                 (file_description['ata_from_url'] == 1)]['url'].values

array(['https://www.saobentoabade.mg.gov.br/site/images/phocadownload/atas/ADENDO_I_ao_Edital_Processo_080_-_Preg%C3%A3o_039.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/atas/Contrato_002_-_processo_012_-_inexigibilidade_002_2020.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/atas/Contrato_001_2020_-_Processo_011_-_Inexigibilidade_001.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/atas/Contrato_003_2020_-_Processo_013_-_Inexigibilidade_003.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/atas/Contrato_083_-_Credencaimento_radiologista.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/atas/TERMO_DE_RATIFICACAO_2110.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/atas/TERMO_DE_RATIFICACAO_1407.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/atas/Termo_de_homologacao_-_Processo_004_-_TP_002.pdf

#### Investigar arquivos que contém a string 'ata' no texto (das 5 primeiras páginas)

In [201]:
print(file_description[(file_description['ata_from_file_content'] == 1)].shape[0],
      'arquivos possuem a string "ata" no texto das 5 primeiras paginas.')

158 arquivos possuem a string "ata" no texto das 5 primeiras paginas.


In [202]:
# Quantos arquivos possuem a string no conteúdo e não na URL?
print(file_description[(file_description['ata_from_file_content'] == 1) &
                       (file_description['ata_from_url'] == 0)]['url'].shape[0],
      'arquivos possuem a string "ata" no texto das 5 primeiras paginas, mas não na URL.')
# Estes arquivos provavelmente são falsos positivos

114 arquivos possuem a string "ata" no texto das 5 primeiras paginas, mas não na URL.


#### Investigar arquivos que contém a string 'ata' no título

In [203]:
file_description[(file_description['ata_from_file_title'] == 1)]['url'].values

array(['https://www.saobentoabade.mg.gov.br/site/images/phocadownload/2020/pl-027-2020-ata-propostas.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/atas/Ata_da_sessao_-_Processo_041_-_Pregao_015_-_sulfite.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/2020/pl-027-2020-ata-habilitacao-e-propostas.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/2020/pl-044-2020_-_Ata_habilitacao.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/atas/ATA_DE_SESS%C3%83O_-_COMPUTADORES.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/atas/ATA_DE_PREGA%CC%83O_-_SERVIC%CC%A7OS_MECA%CC%82NICOS_-_PROCESSO_002_-_PREGA%CC%83O_0021.pdf',
       'https://www.saobentoabade.mg.gov.br/site/phocadownload/editais/2019/atas/pl-061-2019-ata-habilitacao-e-proposta.pdf',
       'https://www.saobentoabade.mg.gov.br/site/images/phocadownload/atas/pl-065-2020-ata.pdf',
       'https://www.s

### Investigar oulines dos arquivos

In [204]:
pdf = PdfFileReader(os.path.join(root_dir, '45af5cc070110fb2b6dc1782daeb1175.pdf'))
print('Number of pages', pdf.getNumPages())
print('Outlines', pdf.getOutlines())
print('Fields', pdf.getFields())
print('Page layout', pdf.getPageLayout())

Number of pages 27
Outlines []
Fields None
Page layout None


#### Seleciona aleatoriamente 20 arquivos de ata e 20 não identificados como atas

In [205]:
# Seleciona aleatoriamente amostras de arquivos
filter_ = file_description[[
    'ata_from_url',
    'ata_from_file_name',
    'ata_from_file_title',
    'ata_from_file_content'
]].sum(axis=1)

# Não foi detectada nenhuma presença da palavra ata
nao_contem_ata = file_description.loc[filter_ == 0, 'file_name'].sample(20, random_state=44).tolist()
# Detectada alguma presença da palavra ata
contem_ata = file_description.loc[filter_ > 0, 'file_name'].sample(40, random_state=44).tolist()
# Onde foi encontrada a palavra ata
file_description.loc[filter_ > 0, ['file_name',
                                   'ata_from_url',
                                   'ata_from_file_name',
                                   'ata_from_file_title',
                                   'ata_from_file_content']].sample(40, random_state=44)

,file_name,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content
4,1cf082dda6251afbebce467c8e0f34bd.pdf,0,0,0,1
204,0357ea1222391c66b5d4e80d85f3e711.pdf,1,0,0,0
71,cce647dc54097635ffd85a078d41b953.pdf,1,1,0,1
251,6327fcf66ddd73347265c83ee0d24c48.pdf,0,0,0,1
140,5466cf275dd5b37f963d6f88f6c89a31.pdf,0,0,0,1
12,5e8ef2b7cea6961012c57acf248d73fb.pdf,1,1,1,1
248,edaaa1d3be7b1c1e5ae2c3b9ad3fd81a.pdf,0,0,0,1
246,8e720b9198a0cbd4acbc1063fced8887.pdf,0,0,0,1
264,dae104ae8394e5ae5b95b95f37639312.pdf,1,1,1,1
104,dcf3a12244fcb5b2ff8c61661ffc8370.pdf,0,0,0,1


#### Copia os arquivos para uma pasta "download"

In [206]:
# Copia os arquivos sorteados para a pasta download
# a pasta será compactada e baixada para análise local
os.makedirs(os.path.join(root_dir, "download"), exist_ok=True)
all_files = contem_ata + nao_contem_ata
for file in all_files:
    src = os.path.join(root_dir, file)
    dst = os.path.join(root_dir, "download", file)    
    shutil.copy(src, dst)

#### Zipa a pasta download

In [207]:
! cd "../data/381-licitacoes-sao-bento-abade/data/files/" && zip -r "download.zip" "download" && cd -

updating: download/ (stored 0%)
updating: download/1cf082dda6251afbebce467c8e0f34bd.pdf (deflated 18%)
updating: download/0357ea1222391c66b5d4e80d85f3e711.pdf (deflated 3%)
updating: download/cce647dc54097635ffd85a078d41b953.pdf (deflated 5%)
updating: download/6327fcf66ddd73347265c83ee0d24c48.pdf (deflated 18%)
updating: download/5466cf275dd5b37f963d6f88f6c89a31.pdf (deflated 19%)
updating: download/5e8ef2b7cea6961012c57acf248d73fb.pdf (deflated 10%)
updating: download/edaaa1d3be7b1c1e5ae2c3b9ad3fd81a.pdf (deflated 9%)
updating: download/8e720b9198a0cbd4acbc1063fced8887.pdf (deflated 10%)
updating: download/dae104ae8394e5ae5b95b95f37639312.pdf (deflated 3%)
updating: download/dcf3a12244fcb5b2ff8c61661ffc8370.pdf (deflated 8%)
updating: download/da6ec38b309b825d53cd142d21cadf4b.pdf (deflated 7%)
updating: download/adecae324a2cf99e56979a845fd87a42.pdf (deflated 5%)
updating: download/ab5f817e540ae109c787e0615b5f0775.pdf (deflated 9%)
updating: download/e9f1d49b2cd9dd197f508e02e1c4fc4c.p

In [208]:
# Move arquivos analisados 
os.makedirs(os.path.join(root_dir, "analisados"), exist_ok=True)
all_files = contem_ata# + nao_contem_ata
for file in all_files:
    src = os.path.join(root_dir, file)
    dst = os.path.join(root_dir, "download", file)    
    shutil.move(src, dst)

In [38]:
sorted(all_files)

['0357ea1222391c66b5d4e80d85f3e711.pdf',
 '16dd8f8ecc2a35f6ffc3668d43b2c70c.pdf',
 '17fa092c0594728d84967f3775b4ed3c.pdf',
 '1a47bd60be7805e841d8b1a8eb4a154b.pdf',
 '1b7d20505c6c9032d71b6180ac7c46a0.pdf',
 '1cf082dda6251afbebce467c8e0f34bd.pdf',
 '1e7a4409640d97b3e9882e0552de63f3.pdf',
 '23779f5e5d06b59950576b58ed6ad205.pdf',
 '28110f12dd6cb2b44387f34ca75a8c47.pdf',
 '286c4aaa06291918525b2cc8b0bffe1f.pdf',
 '4a1b286f67023ebea76f3a99c949926e.pdf',
 '5118cbd1a935d20d2c76a7797ad5ba94.pdf',
 '5466cf275dd5b37f963d6f88f6c89a31.pdf',
 '5e8ef2b7cea6961012c57acf248d73fb.pdf',
 '6327fcf66ddd73347265c83ee0d24c48.pdf',
 '660e4cc412c8a9ffa5f25bfb68b8033f.pdf',
 '6b7ab244fe822aa0f14b550416c169eb.pdf',
 '6f311880cc9cb52baa5af9952795fbd8.pdf',
 '77adc8766143e848fe05619148d744bf.pdf',
 '7872d15f52e9522a5fd7d387e2a97e3e.pdf',
 '85218eb5f8322040be59f1f7ceb69cd7.pdf',
 '8a7e2a1cfa66793b3b77c26074cf5d9f.pdf',
 '8e720b9198a0cbd4acbc1063fced8887.pdf',
 '93edd188687e4b99a7b35920226b07f2.pdf',
 'a3ceec33bdb6cf

In [58]:
file_description['has_ata'] = file_description['ata_from_url'] + file_description['ata_from_file_name'] + file_description['ata_from_file_title'] + file_description['ata_from_file_content']
file_description[file_description.file_name.isin(all_files)][['file_name',
                                                              'ata_from_url',
                                                              'ata_from_file_name',
                                                              'ata_from_file_title',
                                                              'ata_from_file_content']].sort_values(by='file_name')

,file_name,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content
204,0357ea1222391c66b5d4e80d85f3e711.pdf,1,0,0,0
141,16dd8f8ecc2a35f6ffc3668d43b2c70c.pdf,0,0,0,0
47,17fa092c0594728d84967f3775b4ed3c.pdf,0,0,0,1
54,1a47bd60be7805e841d8b1a8eb4a154b.pdf,0,0,0,0
185,1b7d20505c6c9032d71b6180ac7c46a0.pdf,0,0,0,0
4,1cf082dda6251afbebce467c8e0f34bd.pdf,0,0,0,1
75,1e7a4409640d97b3e9882e0552de63f3.pdf,0,0,0,0
53,23779f5e5d06b59950576b58ed6ad205.pdf,0,0,0,1
69,28110f12dd6cb2b44387f34ca75a8c47.pdf,0,0,0,1
242,286c4aaa06291918525b2cc8b0bffe1f.pdf,0,0,0,0


## Olaria

In [18]:
root_dir = "../data/302-licitacoes-olaria/data/files/"
file_description_olaria = pd.read_json(os.path.join(root_dir,"file_description.jsonl"), lines=True)
file_description_olaria = file_description_olaria.loc[file_description_olaria['type'] == 'pdf']
file_description_olaria

,url,crawler_id,instance_id,crawled_at_date,referer,wait_crawler_finish_to_download,time_between_downloads,file_name,type,extracted_files
0,http://www.olaria.mg.gov.br/PMGIRS.pdf,1,160704209604182,2020-12-03 21:35:14.366460,http://www.olaria.mg.gov.br/processos-liciator...,False,NaN,4d53c2fbba114e2e594a3feb6d210c44.pdf,pdf,[coletas/licitacoesolaria/data/csv/4d53c2fbba1...
1,http://www.olaria.mg.gov.br/wp-content/uploads...,1,160704209604182,2020-12-03 21:35:14.366460,http://www.olaria.mg.gov.br/processos-liciator...,False,NaN,fa8663ad5c10f1a7ff08b6344213ff6b.pdf,pdf,[]
2,http://www.olaria.mg.gov.br/wp-content/uploads...,1,160704209604182,2020-12-03 21:35:14.367461,http://www.olaria.mg.gov.br/processos-liciator...,False,NaN,7087aaed7cb40a722df310e1dbd7d881.pdf,pdf,[coletas/licitacoesolaria/data/csv/7087aaed7cb...
7,http://www.olaria.mg.gov.br/wp-content/uploads...,1,160704209604182,2020-12-03 21:36:40.952622,http://www.olaria.mg.gov.br/processos-licitato...,False,NaN,3947c322bad70b9843133a03e1986eea.pdf,pdf,[]
10,http://www.olaria.mg.gov.br/wp-content/uploads...,1,160704209604182,2020-12-03 21:36:40.980546,http://www.olaria.mg.gov.br/processos-licitato...,False,NaN,281f464653c7b5ac70f8c975e467884f.pdf,pdf,[]
13,http://www.olaria.mg.gov.br/wp-content/uploads...,1,160704209604182,2020-12-03 21:36:40.981543,http://www.olaria.mg.gov.br/processos-licitato...,False,NaN,c9a0643b56660ce459adce985157b995.pdf,pdf,[]
14,http://www.olaria.mg.gov.br/wp-content/uploads...,1,160704209604182,2020-12-03 21:36:40.984535,http://www.olaria.mg.gov.br/processos-licitato...,False,NaN,83d54594c1ec629f6138b613a8574e4d.pdf,pdf,[]
16,http://www.olaria.mg.gov.br/wp-content/uploads...,1,160704209604182,2020-12-03 21:36:40.979549,http://www.olaria.mg.gov.br/processos-licitato...,False,NaN,2d0172c38b553df7ceb653c791d59732.pdf,pdf,[]
18,http://www.olaria.mg.gov.br/wp-content/uploads...,1,160704209604182,2020-12-03 21:36:40.955613,http://www.olaria.mg.gov.br/processos-licitato...,False,NaN,0307dcdf9d1d057713a180f61d5daa1f.pdf,pdf,[]
22,http://www.olaria.mg.gov.br/wp-content/uploads...,1,160704209604182,2020-12-03 21:36:40.955613,http://www.olaria.mg.gov.br/processos-licitato...,False,NaN,60a76df5be4434ae1aadfa62c98ad1bd.pdf,pdf,[coletas/licitacoesolaria/data/csv/60a76df5be4...


In [19]:
file_description_olaria['referer'].unique()

array(['http://www.olaria.mg.gov.br/processos-liciatorios/',
       'http://www.olaria.mg.gov.br/processos-licitatorios-2020/',
       'http://www.olaria.mg.gov.br/1253-2/'], dtype=object)

In [20]:
file_description_olaria = identifica_atas_df(file_description_olaria)
file_description_olaria[[
    'ata_from_url',
    'ata_from_file_name',
    'ata_from_file_title',
    'ata_from_file_content'
]].sum()

ata_from_url              0
ata_from_file_name        0
ata_from_file_title      10
ata_from_file_content     0
dtype: int64

In [21]:
file_description_olaria['url'].values

array(['http://www.olaria.mg.gov.br/PMGIRS.pdf',
       'http://www.olaria.mg.gov.br/wp-content/uploads/2017/08/Lei-organica.pdf',
       'http://www.olaria.mg.gov.br/wp-content/uploads/2017/08/Rela%C3%A7%C3%A3o-de-Conv%C3%AAnios.pdf',
       'http://www.olaria.mg.gov.br/wp-content/uploads/2020/07/COMUNICADO-SUSPENS%C3%83O.pdf',
       'http://www.olaria.mg.gov.br/wp-content/uploads/2020/11/resposta-pedido-de-esclarecimento.pdf',
       'http://www.olaria.mg.gov.br/wp-content/uploads/2020/01/Retifica%C3%A7%C3%A3o-de-Edital.pdf',
       'http://www.olaria.mg.gov.br/wp-content/uploads/2020/03/TERMO-DE-RETIFICA%C3%87%C3%83O-1.pdf',
       'http://www.olaria.mg.gov.br/wp-content/uploads/2020/03/RESPOSTA-PEDIDO-DE-ESCLARECIMENTO-ASSINADO.pdf',
       'http://www.olaria.mg.gov.br/wp-content/uploads/2020/07/TERMO-DE-RETIFICA%C3%87%C3%83O.pdf',
       'http://www.olaria.mg.gov.br/wp-content/uploads/2020/10/TERMO-DE-RETIFICA%C3%87%C3%83O-1.pdf',
       'http://www.olaria.mg.gov.br/wp-content/up

In [22]:
file_description_olaria[file_description_olaria['ata_from_file_title'] == 1]['url'].values

array(['http://www.olaria.mg.gov.br/wp-content/uploads/2020/03/TERMO-DE-RETIFICA%C3%87%C3%83O-1.pdf',
       'http://www.olaria.mg.gov.br/wp-content/uploads/2020/07/TERMO-DE-RETIFICA%C3%87%C3%83O.pdf',
       'http://www.olaria.mg.gov.br/wp-content/uploads/2020/10/TERMO-DE-RETIFICA%C3%87%C3%83O-1.pdf',
       'http://www.olaria.mg.gov.br/wp-content/uploads/2020/03/TERMO-DE-RETIFICA%C3%87%C3%83O.pdf',
       'http://www.olaria.mg.gov.br/wp-content/uploads/2020/02/TERMO-DE-RETIFICA%C3%87%C3%83O-1.pdf',
       'http://www.olaria.mg.gov.br/wp-content/uploads/2020/02/TERMO-DE-RETIFICA%C3%87%C3%83O-2.pdf',
       'http://www.olaria.mg.gov.br/wp-content/uploads/2020/08/TERMO-DE-RETIFICA%C3%87%C3%83O.pdf',
       'http://www.olaria.mg.gov.br/wp-content/uploads/2019/12/TERMO-DE-RETIFICA%C3%87%C3%83O.pdf',
       'http://www.olaria.mg.gov.br/wp-content/uploads/2019/03/TERMO-DE-RETIFICA%C3%87%C3%83O-1.pdf',
       'http://www.olaria.mg.gov.br/wp-content/uploads/2019/06/TERMO-DE-RETIFICA%C3%87%C3%

In [25]:
file_description_olaria[file_description_olaria['ata_from_file_title'] == 1].sort_values(by='file_name')['file_name'].values

array(['0307dcdf9d1d057713a180f61d5daa1f.pdf',
       '11ebc58df6363a7c883d54d14f4add65.pdf',
       '60a76df5be4434ae1aadfa62c98ad1bd.pdf',
       '6112b2ba6175d6aadfa40054803ecefd.pdf',
       '7ff5a037761b6c058951fcc5918baebc.pdf',
       '83d54594c1ec629f6138b613a8574e4d.pdf',
       '8856d28873f1463ce4e1c7626148baf0.pdf',
       'a175e2338ba8812da2f1e60192078dee.pdf',
       'c1f985ecfa907506499bf74ff68f0ae7.pdf',
       'fb3f471f27bb73c9437afa3bb3a1b4ca.pdf'], dtype=object)

In [28]:
file_description_olaria.sort_values(by='file_name')[['file_name', 'ata_from_file_title']]

,file_name,ata_from_file_title
121,0142a562899494f636c4671c55baf8f6.pdf,0
169,020010275826404bc551a0ef0628e2dd.pdf,0
18,0307dcdf9d1d057713a180f61d5daa1f.pdf,1
108,0bb101d0438f9eadd1a5ee3e5efbb5ea.pdf,0
103,11a6281a3e73f0dc7ae77fb22fc00315.pdf,0
43,11ebc58df6363a7c883d54d14f4add65.pdf,1
65,1848ab367a2baed7dd27693c6709fe08.pdf,0
163,19a904b26613294dfa0e13a04730f5ef.pdf,0
162,2314b47af4b8d17443c5c6c2fc401024.pdf,0
52,2609c6a6c0bfc91ca0a7f53ed4e17c40.pdf,0


Parece que a cidade de Olaria não possui atas mesmo. Avaliaremos outra cidade.

## Coqueiral

In [31]:
root_dir = "../data/289-licitacoes-coqueiral/data/files/"
file_description_coqueiral = pd.read_json(os.path.join(root_dir,"file_description.jsonl"), lines=True)
file_description_coqueiral = file_description_coqueiral.loc[file_description_coqueiral['type'] == 'pdf']
file_description_coqueiral

,url,file_name,crawler_id,instance_id,crawled_at_date,referer,type,extracted_files
0,https://www.coqueiral.mg.gov.br/public/storage...,2ba855845281501e57212fba9606c873.pdf,1,16076483502732,2020-12-11 01:00:31.840464,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...
1,https://www.coqueiral.mg.gov.br/public/storage...,0b96de38bae73ce0cc9f749d4732ee1a.pdf,1,16076483502732,2020-12-11 01:01:07.498196,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...
2,https://www.coqueiral.mg.gov.br/public/storage...,c452358bc51d7b3995f3b974668edf8b.pdf,1,16076483502732,2020-12-11 01:01:11.998517,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...
3,https://www.coqueiral.mg.gov.br/public/storage...,2b2ea1d5c0576cd25fb9dfdccf67d7bb.pdf,1,16076483502732,2020-12-11 01:01:19.631454,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...
4,https://www.coqueiral.mg.gov.br/public/storage...,fdc2fa2d183e90d75cf6b5cab04f076e.pdf,1,16076483502732,2020-12-11 01:01:24.298670,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...
...,...,...,...,...,...,...,...,...
1523,https://www.coqueiral.mg.gov.br/public/storage...,f0a693a7c838ae635fe1d448b914560a.pdf,1,16076483502732,2020-12-11 04:17:50.657043,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[]
1524,https://www.coqueiral.mg.gov.br/public/storage...,deade3ae8881ee01738e3faaeed17c24.pdf,1,16076483502732,2020-12-11 04:17:53.902169,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[]
1525,https://www.coqueiral.mg.gov.br/public/storage...,c15dbd911335ccf1a1c90565495be000.pdf,1,16076483502732,2020-12-11 04:17:58.592249,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...
1526,https://www.coqueiral.mg.gov.br/public/storage...,4155870ffdb68b75d7d4e407736d7f60.pdf,1,16076483502732,2020-12-11 04:18:04.329898,https://www.coqueiral.mg.gov.br/licitacoes/lic...,pdf,[/home/lucas/Sandbox/coletasbranchnova/coqueir...


In [32]:
file_description_coqueiral = identifica_atas_df(file_description_coqueiral)
file_description_coqueiral[[
    'ata_from_url',
    'ata_from_file_name',
    'ata_from_file_title',
    'ata_from_file_content'
]].sum()

ata_from_url               3
ata_from_file_name         3
ata_from_file_title        0
ata_from_file_content    669
dtype: int64

In [34]:
file_description_coqueiral[[
    'ata_from_url',
    'ata_from_file_name',
    'ata_from_file_title',
    'ata_from_file_content'
]].sum()

ata_from_url               3
ata_from_file_name         3
ata_from_file_title        0
ata_from_file_content    669
dtype: int64

In [33]:
# Seleciona aleatoriamente amostras de arquivos
filter_ = file_description_coqueiral[[
    'ata_from_url',
    'ata_from_file_name',
    'ata_from_file_title',
    'ata_from_file_content'
]].sum(axis=1)

# Não foi detectada nenhuma presença da palavra ata
nao_contem_ata = file_description_coqueiral.loc[filter_ == 0, 'file_name'].sample(20, random_state=44).tolist()
# Detectada alguma presença da palavra ata
contem_ata = file_description_coqueiral.loc[filter_ > 0, 'file_name'].sample(20, random_state=44).tolist()
# Onde foi encontrada a palavra ata
file_description_coqueiral.loc[filter_ > 0, ['file_name',
                                             'ata_from_url',
                                             'ata_from_file_name',
                                             'ata_from_file_title',
                                             'ata_from_file_content']].sample(20, random_state=44)

,file_name,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content
339,a231b4692794fdb36f4e7dd954b1e240.pdf,0,0,0,1
271,4adde75adca4acb2457011da52e998fb.pdf,0,0,0,1
931,7dc160d76221fe80885bbbb1f58d0879.pdf,0,0,0,1
757,ea77c5b25d95afacc03a518f82d3f9ed.pdf,0,0,0,1
636,0a49385614d84618b90f1f2e4174d328.pdf,0,0,0,1
1489,2ec395ad0aae45d48482065864846fe0.pdf,0,0,0,1
903,63f0bb23965d3db31e8001ac818f8b93.pdf,0,0,0,1
535,8e4d577e8e288304b27e66d8c7ee106a.pdf,0,0,0,1
1097,212f10e93e4a4a090891ad2eb047de18.pdf,0,0,0,1
1246,5243120edeaf05ef77756dd29cbba537.pdf,0,0,0,1


In [35]:
# Copia os arquivos sorteados para a pasta download
# a pasta será compactada e baixada para análise local
os.makedirs(os.path.join(root_dir, "download"), exist_ok=True)
all_files = contem_ata + nao_contem_ata
for file in all_files:
    src = os.path.join(root_dir, file)
    dst = os.path.join(root_dir, "download", file)    
    shutil.copy(src, dst)

In [36]:
! cd "../data/289-licitacoes-coqueiral/data/files/" && zip -r "download.zip" "download" && cd -

  adding: download/ (stored 0%)
  adding: download/a231b4692794fdb36f4e7dd954b1e240.pdf (deflated 9%)
  adding: download/4adde75adca4acb2457011da52e998fb.pdf (deflated 7%)
  adding: download/7dc160d76221fe80885bbbb1f58d0879.pdf (deflated 15%)
  adding: download/ea77c5b25d95afacc03a518f82d3f9ed.pdf (deflated 7%)
  adding: download/0a49385614d84618b90f1f2e4174d328.pdf (deflated 7%)
  adding: download/2ec395ad0aae45d48482065864846fe0.pdf (deflated 7%)
  adding: download/63f0bb23965d3db31e8001ac818f8b93.pdf (deflated 7%)
  adding: download/8e4d577e8e288304b27e66d8c7ee106a.pdf (deflated 9%)
  adding: download/212f10e93e4a4a090891ad2eb047de18.pdf (deflated 6%)
  adding: download/5243120edeaf05ef77756dd29cbba537.pdf (deflated 8%)
  adding: download/a4a21d4e3c7a5f21ee6b8aca0c94acc3.pdf (deflated 8%)
  adding: download/239274f51830851f94505ef16076eb33.pdf (deflated 10%)
  adding: download/37c6c8e3bff1841699bee34e7e60a901.pdf (deflated 7%)
  adding: download/8a2737142a40ab87c7bb6d25a865418e.pdf 

In [37]:
sorted(all_files)

['0a49385614d84618b90f1f2e4174d328.pdf',
 '14b14fbd85dc63d510e6bb242422d896.pdf',
 '1b354eb9b59538c25a717391ed12b4d3.pdf',
 '212f10e93e4a4a090891ad2eb047de18.pdf',
 '239274f51830851f94505ef16076eb33.pdf',
 '23cd3730c4b611f89fd7c12f6f4d117b.pdf',
 '2ec395ad0aae45d48482065864846fe0.pdf',
 '37831065c95f2afe6dec028e4eb1a97b.pdf',
 '37c6c8e3bff1841699bee34e7e60a901.pdf',
 '3e6e7a1d0af4d3cee7818f6867dfe746.pdf',
 '3eafd93de40ba50c8eee1b0a087d748b.pdf',
 '4adde75adca4acb2457011da52e998fb.pdf',
 '5243120edeaf05ef77756dd29cbba537.pdf',
 '63f0bb23965d3db31e8001ac818f8b93.pdf',
 '6448e8321106646a53d3c21be1d8fa49.pdf',
 '6f9528e4b108c1054702872edb5fa0a2.pdf',
 '7410f5e54b58e1ebf6f3aa2f8d494549.pdf',
 '7c0b8d46c0389e43c6915856ae7e7d5f.pdf',
 '7dc160d76221fe80885bbbb1f58d0879.pdf',
 '8a2737142a40ab87c7bb6d25a865418e.pdf',
 '8e4d577e8e288304b27e66d8c7ee106a.pdf',
 '95bfcc12c9d51e35df24b0b6d2ba1e09.pdf',
 '9befb4d064f712cad00033c5916736ce.pdf',
 'a231b4692794fdb36f4e7dd954b1e240.pdf',
 'a4a21d4e3c7a5f

In [38]:
file_description_coqueiral[file_description_coqueiral.file_name.isin(all_files)][['file_name',
                                                              'ata_from_url',
                                                              'ata_from_file_name',
                                                              'ata_from_file_title',
                                                              'ata_from_file_content']].sort_values(by='file_name')

,file_name,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content
636,0a49385614d84618b90f1f2e4174d328.pdf,0,0,0,1
819,14b14fbd85dc63d510e6bb242422d896.pdf,0,0,0,1
242,1b354eb9b59538c25a717391ed12b4d3.pdf,0,0,0,0
1097,212f10e93e4a4a090891ad2eb047de18.pdf,0,0,0,1
654,239274f51830851f94505ef16076eb33.pdf,0,0,0,1
937,23cd3730c4b611f89fd7c12f6f4d117b.pdf,0,0,0,0
1489,2ec395ad0aae45d48482065864846fe0.pdf,0,0,0,1
119,37831065c95f2afe6dec028e4eb1a97b.pdf,0,0,0,0
894,37c6c8e3bff1841699bee34e7e60a901.pdf,0,0,0,1
180,3e6e7a1d0af4d3cee7818f6867dfe746.pdf,0,0,0,1


## Pirapetinga

In [121]:
root_dir = "../data/288_licitacoes-pirapetinga/data/files/"
file_description_pirapetinga = pd.read_json(os.path.join(root_dir,"file_description_2.jsonl"), lines=True)
file_description_pirapetinga = file_description_pirapetinga.loc[file_description_pirapetinga['type'] == 'application/pdf']
file_description_pirapetinga

,url,crawler_id,instance_id,crawled_at_date,referer,wait_crawler_finish_to_download,time_between_downloads,file_name,type,extracted_files
0,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 21:58:50.994079,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,bd9cf430f53cfa701f8c7d51e2e0ba2f.pdf,application/pdf,[]
1,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 21:58:50.994526,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,bc0866e315473780ab8d83b14278998f.pdf,application/pdf,[]
2,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 22:00:40.249297,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,c7bd8928fa0ff4856199e1eb2589e8dc.pdf,application/pdf,[]
3,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 22:00:40.249461,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,25bff9c51ff33ab73b3ec2a5c1ae8ea7.pdf,application/pdf,[]
4,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 22:00:40.248959,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,5df4600b8c2d6bfc9fbdb9034a937966.pdf,application/pdf,[]
...,...,...,...,...,...,...,...,...,...,...
1103,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 23:35:39.128086,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,3e081a9c28145f0254998155dc55a05a.pdf,application/pdf,[]
1104,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 23:36:31.837495,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,3fce37da57aef78d7f09770cc9b5593f.pdf,application/pdf,[]
1105,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 23:36:31.837149,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,17c424efc1220e666c3a2a962aa0f716.pdf,application/pdf,[/home/elves/coletores/pirapetinga/data/csv/17...
1106,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 23:36:50.674387,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,450321f5ec33594094414b2e1e71c38d.pdf,application/pdf,[]


Os arquivos listados abaixo são em sua maioria, plantas de projetos arquitetônicos.

In [125]:
file_description_pirapetinga = identifica_atas_df(file_description_pirapetinga)
file_description_pirapetinga[[
    'ata_from_url',
    'ata_from_file_name',
    'ata_from_file_title',
    'ata_from_file_content'
]].sum()

dbd99290d480a36fdcc434d086f70391.pdf
542a278287d8a399752bcfedd924ba5e.pdf
5553f727aa161926b7736565f808dfe0.pdf
9d8cb0b4ab4eae2a85e291387fe47433.pdf
4e2cab94764ce83ad87bb9d2a8bf05e3.pdf
61ba8cd62ce9d68ba40b12f6416b9fde.pdf
4c57c42e007e5cf4e6edf057668e15ec.pdf
a03f8ab5284d9c77a1984302611d9571.pdf
a19becff76c3d306a35fbf0ecb65c655.pdf
cf60687a71a38bdadb522efff242f6f4.pdf
ada30a70f2bbc7b2d031d61596c305fd.pdf
ee119a9db89287f2b67274cc34edcaa7.pdf
c0d892563454aee00c8542244854b346.pdf


e9666b304a3ac80bb39fd57404ac233e.pdf
060f80783ccfe00d02a72b5392074f3c.pdf
2a06a8f9d4ac33708437c96353cb5274.pdf


8438b9a96099354fc63039bd040fbb5d.pdf
c90ac2668b6f7bf96d54fa92b8908f7f.pdf
01d01cf3503d3665b17f1916da262ce7.pdf
99816c024d7cfc7b476a8c8898a32887.pdf
687413b50e44062b9ae31373265a3b34.pdf
f263d166366df40e42b7d929b078de6b.pdf



ata_from_url               0
ata_from_file_name         0
ata_from_file_title        0
ata_from_file_content    169
dtype: int64

In [156]:
# Seleciona aleatoriamente amostras de arquivos
filter_ = file_description_pirapetinga[[
        'ata_from_url',
        'ata_from_file_name',
        'ata_from_file_title',
        'ata_from_file_content'
        ]].sum(axis=1)

# Não foi detectada nenhuma presença da palavra ata
nao_contem_ata = file_description_pirapetinga.loc[filter_ == 0, 'file_name'].sample(20, random_state=44).tolist()
# Detectada alguma presença da palavra ata
contem_ata = file_description_pirapetinga.loc[filter_ > 0, 'file_name'].sample(20, random_state=44).tolist()
# Onde foi encontrada a palavra ata
file_description_pirapetinga.loc[filter_ > 0, ['file_name','ata_from_url','ata_from_file_name','ata_from_file_title','ata_from_file_content']].sample(20, random_state=44)

,file_name,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content
29,a06b3c3b23a6194bd26e1471579bfc1e.pdf,0,0,0,1
618,626436f0c4b32d1a4ddbfcd0f479a75b.pdf,0,0,0,1
672,9d665f6810d01f919372fbc8dada2960.pdf,0,0,0,1
91,208210b68822df4e0a02d88afc9d170a.pdf,0,0,0,1
392,48de864931f8104786dd9fc1efc6ad92.pdf,0,0,0,1
883,b981ccf7574f508b2be741b443f1045f.pdf,0,0,0,1
734,1c2b056fa7cc0c8450ca0f339fc1a237.pdf,0,0,0,1
616,f2ae6b32414474b95cd5d30f256a68f4.pdf,0,0,0,1
773,068f44e883bad231728143f47b15e524.pdf,0,0,0,1
888,45fc4b920b4ecaadd1b005ccf516875f.pdf,0,0,0,1


In [158]:
# Copia os arquivos sorteados para a pasta download
# a pasta será compactada e baixada para análise local
os.makedirs(os.path.join(root_dir, "download"), exist_ok=True)
all_files = contem_ata + nao_contem_ata
for file in all_files:
    src = os.path.join(root_dir, file)
    dst = os.path.join(root_dir, "download", file)    
    shutil.copy(src, dst)

In [159]:
all_files

['a06b3c3b23a6194bd26e1471579bfc1e.pdf',
 '626436f0c4b32d1a4ddbfcd0f479a75b.pdf',
 '9d665f6810d01f919372fbc8dada2960.pdf',
 '208210b68822df4e0a02d88afc9d170a.pdf',
 '48de864931f8104786dd9fc1efc6ad92.pdf',
 'b981ccf7574f508b2be741b443f1045f.pdf',
 '1c2b056fa7cc0c8450ca0f339fc1a237.pdf',
 'f2ae6b32414474b95cd5d30f256a68f4.pdf',
 '068f44e883bad231728143f47b15e524.pdf',
 '45fc4b920b4ecaadd1b005ccf516875f.pdf',
 'd4a2507845e069c1b9e8b1a88e16d636.pdf',
 '7f69a4fa0ca26b791322195e17da9e86.pdf',
 '93395ebecf47a4661813291c89cf3e3e.pdf',
 '503d7d69b04bdc52ed84aaef2d0a0882.pdf',
 '2d8350ef604fccc106b8a164a9da0f11.pdf',
 'da7f01f05048b185d844703ec2cbf82c.pdf',
 '496bf9859877fef6128416d7f68ff3f7.pdf',
 '9719766f3abb916bd3937cd05c81b34e.pdf',
 '6423156f2479b83267beae37664b8559.pdf',
 'c2742d566890cd5546b458e0b549fda1.pdf',
 '6e08183cd385489e397167ebf19f1f9d.pdf',
 'f00654206249cb164c2b354dc6fde732.pdf',
 'e9666b304a3ac80bb39fd57404ac233e.pdf',
 '320242cd2706f9aca2e7f1296a3ff1d7.pdf',
 '0ab92c451b28eb

## Cristais

In [172]:
root_dir = "../data/290-licitacoes-cristais/data/files/"
file_description_cristais = pd.read_json(os.path.join(root_dir,"file_description.jsonl"), lines=True)
file_description_cristais = file_description_cristais.loc[file_description_cristais['type'] == 'pdf;']
file_description_cristais['type'] = 'pdf'
file_description_cristais['file_name'] = file_description_cristais['file_name'].str.replace(";","")
file_description_cristais

,url,file_name,crawler_id,instance_id,crawled_at_date,referer,type,extracted_files
0,https://www.cristais.mg.gov.br/portal/download...,f1a9cef9fa8609613a16d230dbdb7c6f.pdf,2,160762462001740,2020-12-10 18:24:58.997044,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN
1,https://www.cristais.mg.gov.br/portal/download...,6d1cfa73e9217808bf4fc6384294f460.pdf,2,160762462001740,2020-12-10 18:25:02.892016,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN
2,https://www.cristais.mg.gov.br/portal/download...,20ae297764fe60f322fd3ed8e9f27666.pdf,2,160762462001740,2020-12-10 18:25:01.125206,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN
3,https://www.cristais.mg.gov.br/portal/download...,5272e20a7771cead26d153bc040fcc69.pdf,2,160762462001740,2020-12-10 18:25:37.143052,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN
4,https://www.cristais.mg.gov.br/portal/download...,4b58aa2968a032d7be82b0049dc5245a.pdf,2,160762462001740,2020-12-10 18:25:38.310340,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN
...,...,...,...,...,...,...,...,...
1833,https://www.cristais.mg.gov.br/portal/download...,0c5eabd2871ef0c8d52bc28c93a1c211.pdf,2,160762462001740,2020-12-10 20:09:30.192820,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN
1834,https://www.cristais.mg.gov.br/portal/download...,3b7dc6582368510f82bb9e69662f3dfc.pdf,2,160762462001740,2020-12-10 20:09:35.026447,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN
1835,https://www.cristais.mg.gov.br/portal/download...,7834ff090167647a5aee3a737726e141.pdf,2,160762462001740,2020-12-10 20:09:32.828525,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN
1836,https://www.cristais.mg.gov.br/portal/download...,b2ae6cddd299c0e124c28e3aa788a0a1.pdf,2,160762462001740,2020-12-10 20:09:33.253130,https://www.cristais.mg.gov.br/portal/editais/...,pdf,NaN


In [175]:
file_description_cristais = identifica_atas_df(file_description_cristais)
file_description_cristais[[
    'ata_from_url',
    'ata_from_file_name',
    'ata_from_file_title',
    'ata_from_file_content'
]].sum()

678ebb1cc0b5b9ca358a2b14450c840a.pdf: Cannot read an empty file
827a398f255b73d083d073b4e3aed5cf.pdf: Cannot read an empty file
8ce63c93769d62d1c716f27b4e7dbf10.pdf: Could not find xref table at specified location
45cc0a2aff268eb41ccb07427c46d90f.pdf: Could not find xref table at specified location
dfc237ffd10c1b6bc62fe5aac5137977.pdf: Could not find xref table at specified location
d4e4037f2275159b54f08fa7d66ec028.pdf: Could not find xref table at specified location


ata_from_url               0
ata_from_file_name         0
ata_from_file_title      133
ata_from_file_content    745
dtype: int64

In [176]:
# Seleciona aleatoriamente amostras de arquivos
filter_ = file_description_cristais[[
        'ata_from_url',
        'ata_from_file_name',
        'ata_from_file_title',
        'ata_from_file_content'
        ]].sum(axis=1)

# Não foi detectada nenhuma presença da palavra ata
nao_contem_ata = file_description_cristais.loc[filter_ == 0, 'file_name'].sample(20, random_state=44).tolist()
# Detectada alguma presença da palavra ata
contem_ata = file_description_cristais.loc[filter_ > 0, 'file_name'].sample(20, random_state=44).tolist()
# Onde foi encontrada a palavra ata
file_description_cristais.loc[filter_ > 0, ['file_name','ata_from_url','ata_from_file_name','ata_from_file_title','ata_from_file_content']].sample(20, random_state=44)

,file_name,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content
1066,ab507c03faa869c93ce709b37eb27e39.pdf,0,0,0,1
431,d08d18740ab8fd1b820e9652128eafb2.pdf,0,0,0,1
1776,29cd60cfbd728232c60cb2c020fb01d7.pdf,0,0,0,1
1204,1c371781f2f1a7b30f0c7f8728e8b42a.pdf,0,0,0,1
1041,62c40d905e2d4fd5764caa0398cb143b.pdf,0,0,0,1
241,9ff316e201bfde65d1dbfffdb3e62988.pdf,0,0,0,1
1242,aed27e6141e4bafe8ea18271d8faafbd.pdf,0,0,0,1
839,83b5a0e7afaf7a6d79b6d93e7ec0272a.pdf,0,0,0,1
451,2db27857260eb235dea6d924f2a99bf2.pdf,0,0,1,1
159,3f85d81015d9774e9ba6239d53188307.pdf,0,0,0,1


In [184]:
contem_ata

['ab507c03faa869c93ce709b37eb27e39.pdf',
 'd08d18740ab8fd1b820e9652128eafb2.pdf',
 '29cd60cfbd728232c60cb2c020fb01d7.pdf',
 '1c371781f2f1a7b30f0c7f8728e8b42a.pdf',
 '62c40d905e2d4fd5764caa0398cb143b.pdf',
 '9ff316e201bfde65d1dbfffdb3e62988.pdf',
 'aed27e6141e4bafe8ea18271d8faafbd.pdf',
 '83b5a0e7afaf7a6d79b6d93e7ec0272a.pdf',
 '2db27857260eb235dea6d924f2a99bf2.pdf',
 '3f85d81015d9774e9ba6239d53188307.pdf',
 'ff3f87fc145fd2c43bee6314884750c2.pdf',
 '465d432f103c446cfb40c39585a0474f.pdf',
 'b194b2892e8d5bb7a20da85e5370287b.pdf',
 '5eecbe8a1574aa43723fb6f956b74fe0.pdf',
 'fa9ce46c17c6192ff2c7ea18626f4f84.pdf',
 'cfbecb34991f3ae2e61be7e26cee5d99.pdf',
 'bf108ccf7cdbb721b16078aba522514b.pdf',
 '33d26a45a3770f3ae1618334ddf6d209.pdf',
 'cf4e17de5470875c1ba4a83b94fefe3b.pdf',
 'b4caadc0a18961138f84eb2e66b07967.pdf']

In [177]:
# Copia os arquivos sorteados para a pasta download
# a pasta será compactada e baixada para análise local
os.makedirs(os.path.join(root_dir, "download"), exist_ok=True)
all_files = contem_ata + nao_contem_ata
for file in all_files:
    src = os.path.join(root_dir, file)
    dst = os.path.join(root_dir, "download", file)    
    shutil.copy(src, dst)

In [181]:
os.path.join(root_dir, "download.zip")

'../data/290-licitacoes-cristais/data/files/download.zip'

## Passa Vinte

In [26]:
root_dir = "../data/304-licitacoes-passa-vinte/data/files/"
file_description_passa_vinte = pd.read_json(os.path.join(root_dir,"file_description.jsonl"), lines=True)
file_description_passa_vinte = file_description_passa_vinte.loc[file_description_passa_vinte['type'] == 'pdf']
file_description_passa_vinte

,url,file_name,crawler_id,instance_id,crawled_at_date,referer,type,time_between_downloads,extracted_files
0,https://www.passavinte.mg.gov.br/download/down...,4547a6d7529ac939fcbc5b579bd64c40.pdf,5,160746828499708,2020-12-08 23:06:39.720399,https://www.passavinte.mg.gov.br/processo-no-1...,pdf,NaN,[licitacoes_passa_vinte/data/csv/4547a6d7529ac...
1,https://www.passavinte.mg.gov.br/download/down...,0b1985fc3b17b7401eaeac6d6b95ca2c.pdf,5,160746828499708,2020-12-08 23:07:58.806685,https://www.passavinte.mg.gov.br/processo-20-2...,pdf,NaN,[licitacoes_passa_vinte/data/csv/0b1985fc3b17b...
2,https://www.passavinte.mg.gov.br/download/down...,83fa2991f65f1e7d7df105b85b12fc04.pdf,5,160746828499708,2020-12-08 23:08:23.468321,https://www.passavinte.mg.gov.br/processo-lici...,pdf,NaN,[licitacoes_passa_vinte/data/csv/83fa2991f65f1...
3,https://www.passavinte.mg.gov.br/download/down...,cd019d89436cbe6884ce0c6a166ea9e1.pdf,5,160746828499708,2020-12-08 23:13:15.976820,https://www.passavinte.mg.gov.br/download/down...,pdf,NaN,[licitacoes_passa_vinte/data/csv/cd019d89436cb...
4,https://www.passavinte.mg.gov.br/download/down...,074d45ccafdf94b6e509d8259dfcd295.pdf,5,160746828499708,2020-12-08 23:14:18.830225,https://www.passavinte.mg.gov.br/download/down...,pdf,NaN,[licitacoes_passa_vinte/data/csv/074d45ccafdf9...
...,...,...,...,...,...,...,...,...,...
406,https://www.passavinte.mg.gov.br/download/down...,53e47bbe6410616c75f15adf45d68fc5.pdf,5,160751772286184,2020-12-09 18:03:05.250501,https://www.passavinte.mg.gov.br/download/down...,pdf,NaN,[licitacoes_passa_vinte/data/csv/53e47bbe64106...
408,https://www.passavinte.mg.gov.br/download/cliq...,5e9d440f470f8d27ecb083f1831e62db.pdf,5,160751772286184,2020-12-09 18:08:03.759279,https://www.passavinte.mg.gov.br/download/cliq...,pdf,NaN,[]
409,https://www.passavinte.mg.gov.br/download/down...,b00eff4906a6e74be9d9a72f49c0df1e.pdf,5,160751772286184,2020-12-09 18:08:44.089100,https://www.passavinte.mg.gov.br/download/down...,pdf,NaN,[licitacoes_passa_vinte/data/csv/b00eff4906a6e...
410,https://www.passavinte.mg.gov.br/download/down...,90cae1507d73f1fd8c47cf4ad590ffb1.pdf,5,160751772286184,2020-12-09 18:09:14.962170,https://www.passavinte.mg.gov.br/download/down...,pdf,NaN,[licitacoes_passa_vinte/data/csv/90cae1507d73f...


In [27]:
file_description_passa_vinte = identifica_atas_df(file_description_passa_vinte)
file_description_passa_vinte[[
    'ata_from_url',
    'ata_from_file_name',
    'ata_from_file_title',
    'ata_from_file_content'
]].sum()

ata_from_url               2
ata_from_file_name         0
ata_from_file_title        0
ata_from_file_content    144
dtype: int64

In [29]:
# Seleciona aleatoriamente amostras de arquivos
filter_ = file_description_passa_vinte[[
        'ata_from_url',
        'ata_from_file_name',
        'ata_from_file_title',
        'ata_from_file_content'
        ]].sum(axis=1)

# Não foi detectada nenhuma presença da palavra ata
nao_contem_ata = file_description_passa_vinte.loc[filter_ == 0, 'file_name'].sample(20, random_state=44).tolist()
# Detectada alguma presença da palavra ata
contem_ata = file_description_passa_vinte.loc[filter_ > 0, 'file_name'].sample(20, random_state=44).tolist()
# Onde foi encontrada a palavra ata
file_description_passa_vinte.loc[filter_ > 0, ['file_name',
                                               'ata_from_url',
                                               'ata_from_file_name',
                                               'ata_from_file_title',
                                               'ata_from_file_content']].sample(20, random_state=44)

,file_name,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content
382,d418f9a1167a945f81c0d5a0b5795955.pdf,0,0,0,1
138,f0c17cbcf07e8c2b23fd66692603cce3.pdf,0,0,0,1
158,68bd974c7582e78932e167e71e97a4c9.pdf,0,0,0,1
156,e67156f2f8cd08f826e19320962837dd.pdf,0,0,0,1
276,e331a62002cd1dadd18054d9c3d7de26.pdf,0,0,0,1
100,efba000a7dcbafe9e92a77942ce54f06.pdf,0,0,0,1
209,3ca23150b771c4b35ce874552bc5d6d2.pdf,0,0,0,1
215,90d907b9b1c49cd29d6475c72cc384c6.pdf,0,0,0,1
313,6119b9c2d58a489fbf0a3c6459af692b.pdf,0,0,0,1
290,e4e6fe425796b4331b1504aca21317bc.pdf,0,0,0,1


In [30]:
# Copia os arquivos sorteados para a pasta download
# a pasta será compactada e baixada para análise local
os.makedirs(os.path.join(root_dir, "download"), exist_ok=True)
all_files = contem_ata + nao_contem_ata
for file in all_files:
    src = os.path.join(root_dir, file)
    dst = os.path.join(root_dir, "download", file)    
    shutil.copy(src, dst)

In [31]:
! cd "../data/304-licitacoes-passa-vinte/data/files/" && zip -r "download/download.zip" "download" && cd -

  adding: download/ (stored 0%)
  adding: download/d418f9a1167a945f81c0d5a0b5795955.pdf (deflated 9%)
  adding: download/f0c17cbcf07e8c2b23fd66692603cce3.pdf (deflated 14%)
  adding: download/68bd974c7582e78932e167e71e97a4c9.pdf (deflated 17%)
  adding: download/e67156f2f8cd08f826e19320962837dd.pdf (deflated 6%)
  adding: download/e331a62002cd1dadd18054d9c3d7de26.pdf (deflated 2%)
  adding: download/efba000a7dcbafe9e92a77942ce54f06.pdf (deflated 3%)
  adding: download/3ca23150b771c4b35ce874552bc5d6d2.pdf (deflated 9%)
  adding: download/90d907b9b1c49cd29d6475c72cc384c6.pdf (deflated 10%)
  adding: download/6119b9c2d58a489fbf0a3c6459af692b.pdf (deflated 14%)
  adding: download/e4e6fe425796b4331b1504aca21317bc.pdf (deflated 3%)
  adding: download/47469eda3889f7ffb045290e8072f583.pdf (deflated 14%)
  adding: download/414f191b7e906bd7baa4888d1636a073.pdf (deflated 2%)
  adding: download/344f6f0d3b92073099fd14ad20da2400.pdf (deflated 12%)
  adding: download/90cae1507d73f1fd8c47cf4ad590ffb1.

In [40]:
# Verifica tipo dos arquivos que contem ata
file_description_passa_vinte[file_description_passa_vinte['file_name'].isin(contem_ata)][['file_name',
                                                                                           'ata_from_url',
                                                                                           'ata_from_file_name',
                                                                                           'ata_from_file_title',
                                                                                           'ata_from_file_content']]

,file_name,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content
15,414f191b7e906bd7baa4888d1636a073.pdf,0,0,0,1
36,d3193a43ddd0ea8d5b186efe3c2f148e.pdf,0,0,0,1
100,efba000a7dcbafe9e92a77942ce54f06.pdf,0,0,0,1
122,cec791ee5748ce1f10571990c1715d16.pdf,0,0,0,1
138,f0c17cbcf07e8c2b23fd66692603cce3.pdf,0,0,0,1
141,47469eda3889f7ffb045290e8072f583.pdf,0,0,0,1
156,e67156f2f8cd08f826e19320962837dd.pdf,0,0,0,1
158,68bd974c7582e78932e167e71e97a4c9.pdf,0,0,0,1
160,b83d625748d46581e38522f2b7521185.pdf,0,0,0,1
209,3ca23150b771c4b35ce874552bc5d6d2.pdf,0,0,0,1


## Arantina

In [3]:
root_dir = "../data/306-licitacoes-arantina/data/files/"
file_description_arantina = pd.read_json(os.path.join(root_dir,"file_description.jsonl"), lines=True)
file_description_arantina = file_description_arantina.loc[file_description_arantina['type'] == 'pdf']
file_description_arantina

,url,crawler_id,instance_id,crawled_at_date,referer,wait_crawler_finish_to_download,time_between_downloads,file_name,type
0,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:30:30.175044,https://www.arantina.mg.gov.br/licitacao2020,False,1,11911196c980f90a1abc6bdad3a63973.pdf,pdf
1,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:30:34.236158,https://www.arantina.mg.gov.br/leis2020,False,1,be439ad0f779d00dbf96310c07c58abb.pdf,pdf
2,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:30:34.237156,https://www.arantina.mg.gov.br/leis2020,False,1,6630277a946046b0e70cbdbf314d51f4.pdf,pdf
3,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:30:32.706535,https://www.arantina.mg.gov.br/copia-dispensa-...,False,1,c387044bcb423b95c007a43208102e8d.pdf,pdf
4,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:30:34.240611,https://www.arantina.mg.gov.br/leis2020,False,1,aeea94794d0f556d5c97ff93bf76e18c.pdf,pdf
...,...,...,...,...,...,...,...,...,...
978,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:31:48.308021,https://www.arantina.mg.gov.br/copia-pregao-2016,False,1,2b509d3713d3d53a5dda605e50a2a461.pdf,pdf
979,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:32:19.651331,https://www.arantina.mg.gov.br/copia-pregao-2017,False,1,a394f5dadda36a930be8b13fad91b33f.pdf,pdf
980,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:32:19.649335,https://www.arantina.mg.gov.br/copia-pregao-2017,False,1,5b5c51d32873c63ac641bfd2db34829d.pdf,pdf
981,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:31:07.810052,https://www.arantina.mg.gov.br/dispensalicitac...,False,1,6842508209456f43b63cb6ac4fff61fe.pdf,pdf


In [4]:
# Procurando duplicatas (url e file_name)
file_description_arantina[file_description_arantina.duplicated(subset=['url', 'file_name'])].head()

,url,crawler_id,instance_id,crawled_at_date,referer,wait_crawler_finish_to_download,time_between_downloads,file_name,type
39,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:30:34.236773,https://www.arantina.mg.gov.br/leis2020,False,1,11911196c980f90a1abc6bdad3a63973.pdf,pdf
60,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:30:36.440537,https://www.arantina.mg.gov.br/tomadadepreco2020,False,1,11911196c980f90a1abc6bdad3a63973.pdf,pdf
62,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:30:32.710968,https://www.arantina.mg.gov.br/copia-dispensa-...,False,1,11911196c980f90a1abc6bdad3a63973.pdf,pdf
72,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:30:41.202029,https://www.arantina.mg.gov.br/copia-licitacao...,False,1,11911196c980f90a1abc6bdad3a63973.pdf,pdf
73,https://1bf55a46-c691-4e03-915a-501a5d7f5ffb.f...,2,160566302592761,2020-11-17 22:30:38.787060,https://www.arantina.mg.gov.br/inexigibilidade...,False,1,11911196c980f90a1abc6bdad3a63973.pdf,pdf


A coleta do município de Arantina possui arquivos duplicados (com a mesma `url` e `file_name`). Como o `referer` é diferente, provavelmente é possível chegar a este mesmo arquivo a partir de páginas diferentes.

In [5]:
# Removendo duplicatas e mantendo somente a primeira ocorrência
file_description_arantina.drop_duplicates(subset=['url', 'file_name'], keep='first', inplace=True)

# Tamanho do dataframe final
file_description_arantina.shape

(937, 9)

In [6]:
file_description_arantina = identifica_atas_df(file_description_arantina)
file_description_arantina[[
    'ata_from_url',
    'ata_from_file_name',
    'ata_from_file_title',
    'ata_from_file_content'
]].sum()

ata_from_url               0
ata_from_file_name         0
ata_from_file_title        0
ata_from_file_content    160
dtype: int64

In [8]:
# Seleciona aleatoriamente amostras de arquivos
filter_ = file_description_arantina[[
        'ata_from_url',
        'ata_from_file_name',
        'ata_from_file_title',
        'ata_from_file_content'
        ]].sum(axis=1)

# Não foi detectada nenhuma presença da palavra ata
nao_contem_ata = file_description_arantina.loc[filter_ == 0, 'file_name'].sample(20, random_state=44).tolist()
# Detectada alguma presença da palavra ata
contem_ata = file_description_arantina.loc[filter_ > 0, 'file_name'].sample(20, random_state=44).tolist()
# Onde foi encontrada a palavra ata
file_description_arantina.loc[filter_ > 0, ['file_name',
                                               'ata_from_url',
                                               'ata_from_file_name',
                                               'ata_from_file_title',
                                               'ata_from_file_content']].sample(20, random_state=44)

,file_name,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content
489,82ba10429dbbbb06ae36e5865037a4c1.pdf,0,0,0,1
96,c0021dd9bce3969ec38409c3f7a838e4.pdf,0,0,0,1
919,93618ff8e5fbe25a84ef3628a769028b.pdf,0,0,0,1
982,6ce4b1e5b9c71b48cc38ac8fb4a82f4c.pdf,0,0,0,1
154,befb6749744a1525dc0dd253fb3fd4ee.pdf,0,0,0,1
599,5277e6b5a242a510d0e299782b917d42.pdf,0,0,0,1
307,946e108f71188c8a464b94367d05703e.pdf,0,0,0,1
765,42c58cf0299eae972742d95076ca6ed2.pdf,0,0,0,1
302,0b7247daadf2f44fb6ea27c684b0c7ab.pdf,0,0,0,1
751,f27cadd6414886e20e95d4bd0b313bec.pdf,0,0,0,1


In [9]:
# Copia os arquivos sorteados para a pasta download
# a pasta será compactada e baixada para análise local
os.makedirs(os.path.join(root_dir, "download"), exist_ok=True)
all_files = contem_ata + nao_contem_ata
for file in all_files:
    src = os.path.join(root_dir, file)
    dst = os.path.join(root_dir, "download", file) 
    shutil.copy(src, dst)

In [10]:
! cd "../data/306-licitacoes-arantina/data/files/" && zip -r "download/download.zip" "download" && cd -

  adding: download/ (stored 0%)
  adding: download/82ba10429dbbbb06ae36e5865037a4c1.pdf (deflated 8%)
  adding: download/c0021dd9bce3969ec38409c3f7a838e4.pdf (deflated 9%)
  adding: download/93618ff8e5fbe25a84ef3628a769028b.pdf (deflated 11%)
  adding: download/6ce4b1e5b9c71b48cc38ac8fb4a82f4c.pdf (deflated 9%)
  adding: download/befb6749744a1525dc0dd253fb3fd4ee.pdf (deflated 8%)
  adding: download/5277e6b5a242a510d0e299782b917d42.pdf (deflated 10%)
  adding: download/946e108f71188c8a464b94367d05703e.pdf (deflated 9%)
  adding: download/42c58cf0299eae972742d95076ca6ed2.pdf (deflated 8%)
  adding: download/0b7247daadf2f44fb6ea27c684b0c7ab.pdf (deflated 8%)
  adding: download/f27cadd6414886e20e95d4bd0b313bec.pdf (deflated 9%)
  adding: download/aafa20ab5ed7652901316a05c0d164ef.pdf (deflated 7%)
  adding: download/3021555df4a51338cc9a710b083289ad.pdf (deflated 8%)
  adding: download/87b061cd7e96cd8de902dab42f868ed3.pdf (deflated 6%)
  adding: download/8c654035f2cd36f265fb94b61ff2dcbe.pdf 

In [11]:
# Verifica tipo dos arquivos que contem ata
file_description_arantina[file_description_arantina['file_name'].isin(contem_ata)][['file_name',
                                                                                           'ata_from_url',
                                                                                           'ata_from_file_name',
                                                                                           'ata_from_file_title',
                                                                                           'ata_from_file_content']]

,file_name,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content
96,c0021dd9bce3969ec38409c3f7a838e4.pdf,0,0,0,1
154,befb6749744a1525dc0dd253fb3fd4ee.pdf,0,0,0,1
302,0b7247daadf2f44fb6ea27c684b0c7ab.pdf,0,0,0,1
307,946e108f71188c8a464b94367d05703e.pdf,0,0,0,1
333,e8867b38d7f6ab6e21148963b3f781c4.pdf,0,0,0,1
418,9b471cc446b25c633e77944bcbbd7d84.pdf,0,0,0,1
488,23745b4a3bc79647d02f058658dce614.pdf,0,0,0,1
489,82ba10429dbbbb06ae36e5865037a4c1.pdf,0,0,0,1
497,8c654035f2cd36f265fb94b61ff2dcbe.pdf,0,0,0,1
599,5277e6b5a242a510d0e299782b917d42.pdf,0,0,0,1


# Ijaci

In [4]:
root_dir = "../data/353-licitacoes-ijaci/data/files/"
file_description_ijaci = pd.read_json(os.path.join(root_dir,"file_description.jsonl"), lines=True)
file_description_ijaci = file_description_ijaci.loc[file_description_ijaci['type'] == 'pdf']
file_description_ijaci

,url,file_name,crawler_id,instance_id,crawled_at_date,referer,type,time_between_downloads,extracted_files
0,http://ijacimg.web21f14.uni5.net/licitacao/adm...,f1c2a67ca57a8f5df13f8565cd4eb306.pdf,2,160752576794909,2020-12-09 11:56:46.360628,http://ijacimg.web21f14.uni5.net/licitacao/con...,pdf,NaN,[]
1,http://ijacimg.web21f14.uni5.net/licitacao/adm...,fc4ffae99918d4175dc5d1a69d8e74e3.pdf,2,160752576794909,2020-12-09 11:57:12.237104,http://ijacimg.web21f14.uni5.net/licitacao/con...,pdf,NaN,[]
2,http://ijacimg.web21f14.uni5.net/licitacao/adm...,b1e217665d6826ffc60261120acc0bc2.pdf,2,160752576794909,2020-12-09 11:57:20.287784,http://ijacimg.web21f14.uni5.net/licitacao/con...,pdf,NaN,[coletas/licitacoesijaci/data/csv/b1e217665d68...
3,http://ijacimg.web21f14.uni5.net/licitacao/adm...,a19ea49327a5a15fb50d2582d67940d8.pdf,2,160752576794909,2020-12-09 11:57:34.423802,http://ijacimg.web21f14.uni5.net/licitacao/con...,pdf,NaN,[]
4,http://ijacimg.web21f14.uni5.net/licitacao/adm...,c59117c3c4d5cc84279ed3138be32732.pdf,2,160752576794909,2020-12-09 11:57:43.347995,http://ijacimg.web21f14.uni5.net/licitacao/con...,pdf,NaN,[]
...,...,...,...,...,...,...,...,...,...
450,http://ijacimg.web21f14.uni5.net/licitacao/adm...,416e0ec0706bba98e5598b387b328d21.pdf,2,160752576794909,2020-12-09 13:09:19.308455,http://ijacimg.web21f14.uni5.net/licitacao/con...,pdf,NaN,[]
451,http://ijacimg.web21f14.uni5.net/licitacao/adm...,9d9ec4d391b27a2cce00d13df221af3a.pdf,2,160752576794909,2020-12-09 13:09:26.361195,http://ijacimg.web21f14.uni5.net/licitacao/con...,pdf,NaN,[coletas/licitacoesijaci/data/csv/9d9ec4d391b2...
452,http://ijacimg.web21f14.uni5.net/licitacao/adm...,4ff015a5c06934e3d15db378aa8fd931.pdf,2,160752576794909,2020-12-09 13:09:37.602159,http://ijacimg.web21f14.uni5.net/licitacao/con...,pdf,NaN,[]
453,http://ijacimg.web21f14.uni5.net/licitacao/adm...,025203d5356694d8cce2d4b988d5541c.pdf,2,160752576794909,2020-12-09 13:09:45.560149,http://ijacimg.web21f14.uni5.net/licitacao/con...,pdf,NaN,[]


In [10]:
file_description_ijaci = identifica_atas_df(file_description_ijaci)
file_description_ijaci[[
    'ata_from_url',
    'ata_from_file_name',
    'ata_from_file_title',
    'ata_from_file_content'
]].sum()

ata_from_url             31
ata_from_file_name       31
ata_from_file_title       1
ata_from_file_content    33
dtype: int64

In [16]:
# Seleciona aleatoriamente amostras de arquivos
filter_ = file_description_ijaci[[
    'ata_from_url',
    'ata_from_file_name',
    'ata_from_file_title',
    'ata_from_file_content'
]].sum(axis=1)

# Não foi detectada nenhuma presença da palavra ata
nao_contem_ata = file_description_ijaci.loc[filter_ == 0, 'file_name'].sample(20, random_state=44).tolist()
# Detectada alguma presença da palavra ata
contem_ata = file_description_ijaci.loc[filter_ > 0, 'file_name'].sample(20, random_state=44).tolist()
# Onde foi encontrada a palavra ata
file_description_ijaci.loc[filter_ > 0, ['file_name',
                                             'ata_from_url',
                                             'ata_from_file_name',
                                             'ata_from_file_title',
                                             'ata_from_file_content']].sample(20, random_state=44)

,file_name,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content
354,093e8c2a45aa73c9a054ec37400208ef.pdf,1,1,0,0
21,4d652b7d7b5f7e64b551c1ca87365dd6.pdf,0,0,0,1
0,f1c2a67ca57a8f5df13f8565cd4eb306.pdf,1,1,0,0
416,a636c085b63fd13215e8539ad7959803.pdf,1,1,0,0
20,af60eff022ab670d1b486ecd8171cd4e.pdf,0,0,0,1
102,fdf7e4463c02b0e2fbc01b82a1238e31.pdf,0,0,0,1
19,e8238626be4e8e167cda486f3a119cf1.pdf,0,0,0,1
404,9ab800e3b88a560a9439b2d76ea50eb1.pdf,1,1,0,0
121,ef9f167ccb4a02f60b363b4705bf2832.pdf,1,1,0,0
53,2fdef51a0d97860f84ac01ef0e6a0c0d.pdf,0,0,0,1


In [17]:
# Copia os arquivos sorteados para a pasta download
# a pasta será compactada e baixada para análise local
os.makedirs(os.path.join(root_dir, "download"), exist_ok=True)
all_files = contem_ata + nao_contem_ata
for file in all_files:
    src = os.path.join(root_dir, file)
    dst = os.path.join(root_dir, "download", file)    
    shutil.copy(src, dst)

In [18]:
! cd "../data/353-licitacoes-ijaci/data/files/" && zip -r "download/download.zip" "download" && cd -

  adding: download/ (stored 0%)
  adding: download/093e8c2a45aa73c9a054ec37400208ef.pdf (deflated 3%)
  adding: download/4d652b7d7b5f7e64b551c1ca87365dd6.pdf (deflated 8%)
  adding: download/f1c2a67ca57a8f5df13f8565cd4eb306.pdf (deflated 3%)
  adding: download/a636c085b63fd13215e8539ad7959803.pdf (deflated 3%)
  adding: download/af60eff022ab670d1b486ecd8171cd4e.pdf (deflated 7%)
  adding: download/fdf7e4463c02b0e2fbc01b82a1238e31.pdf (deflated 10%)
  adding: download/e8238626be4e8e167cda486f3a119cf1.pdf (deflated 7%)
  adding: download/9ab800e3b88a560a9439b2d76ea50eb1.pdf (deflated 3%)
  adding: download/ef9f167ccb4a02f60b363b4705bf2832.pdf (deflated 4%)
  adding: download/2fdef51a0d97860f84ac01ef0e6a0c0d.pdf (deflated 4%)
  adding: download/ffc6d89d330a0a2ea66183d7377adbac.pdf (deflated 3%)
  adding: download/68bb96f38f82b6cb1f1c740a32b23bc1.pdf (deflated 4%)
  adding: download/e00388a628eaa3e9d139e50fab171d5e.pdf (deflated 4%)
  adding: download/a0c33fcda4eb23a5d19756124efb2804.pdf (

In [19]:
# Verifica tipo dos arquivos que contem ata
file_description_ijaci[file_description_ijaci['file_name'].isin(contem_ata)][['file_name',
                                                                               'ata_from_url',
                                                                               'ata_from_file_name',
                                                                               'ata_from_file_title',
                                                                               'ata_from_file_content']]

,file_name,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content
0,f1c2a67ca57a8f5df13f8565cd4eb306.pdf,1,1,0,0
14,82cc1ed49eb0a4811057913f75d2a248.pdf,0,0,0,1
19,e8238626be4e8e167cda486f3a119cf1.pdf,0,0,0,1
20,af60eff022ab670d1b486ecd8171cd4e.pdf,0,0,0,1
21,4d652b7d7b5f7e64b551c1ca87365dd6.pdf,0,0,0,1
30,68bb96f38f82b6cb1f1c740a32b23bc1.pdf,0,0,0,1
33,c6f2ea831cffa3b3499bf014765354f1.pdf,0,0,0,1
37,c65d6dae65c2a16410a109fa142db068.pdf,1,1,0,1
40,a0c33fcda4eb23a5d19756124efb2804.pdf,0,0,0,1
42,e00388a628eaa3e9d139e50fab171d5e.pdf,0,0,0,1


In [20]:
all_files

['093e8c2a45aa73c9a054ec37400208ef.pdf',
 '4d652b7d7b5f7e64b551c1ca87365dd6.pdf',
 'f1c2a67ca57a8f5df13f8565cd4eb306.pdf',
 'a636c085b63fd13215e8539ad7959803.pdf',
 'af60eff022ab670d1b486ecd8171cd4e.pdf',
 'fdf7e4463c02b0e2fbc01b82a1238e31.pdf',
 'e8238626be4e8e167cda486f3a119cf1.pdf',
 '9ab800e3b88a560a9439b2d76ea50eb1.pdf',
 'ef9f167ccb4a02f60b363b4705bf2832.pdf',
 '2fdef51a0d97860f84ac01ef0e6a0c0d.pdf',
 'ffc6d89d330a0a2ea66183d7377adbac.pdf',
 '68bb96f38f82b6cb1f1c740a32b23bc1.pdf',
 'e00388a628eaa3e9d139e50fab171d5e.pdf',
 'a0c33fcda4eb23a5d19756124efb2804.pdf',
 'c6f2ea831cffa3b3499bf014765354f1.pdf',
 '82cc1ed49eb0a4811057913f75d2a248.pdf',
 'c65d6dae65c2a16410a109fa142db068.pdf',
 '8ae714adac4129d5d774344764efb082.pdf',
 'be1870ba6102a34a05ec0a50a7e53c56.pdf',
 'b38ccdde54d7c1ba6cb0d2591173c8e0.pdf',
 '21052d8c4d97033b42e10ac5344b0610.pdf',
 '15ce0e9e01998171b1a99ecfdc78c271.pdf',
 '4525fa1925589187eea2f2dc82fdc279.pdf',
 '98b10427b1864a8771997aa8d4be7420.pdf',
 'c2cbf425a5d7e0

# Perdões

In [16]:
root_dir = "../data/br.gov.mg.perdoes/159735468355964/files/"
file_description_perdoes = pd.read_json(os.path.join(root_dir,"file_description.jsonl"), lines=True)
file_description_perdoes

,file_name,url,crawler_id,type,crawled_at_date
0,40c3ce71ebe240147ecb540e5b1fe493.pdf,https://www.perdoes.mg.gov.br/flex/cms_downloa...,159735468355964,b'application/pdf',2020-08-13 21:38:13.486415
1,825e97e71b76cd16d3b4724c3efed0fd.pdf,https://www.perdoes.mg.gov.br/flex/cms_downloa...,159735468355964,b'application/pdf',2020-08-13 21:38:14.406434
2,ebf202a8364db3cfc7d4c4b0c50cad94.pdf,https://www.perdoes.mg.gov.br/flex/cms_downloa...,159735468355964,b'application/pdf',2020-08-13 21:38:18.196042
3,d97b229ddd2ef3e6e287f63a454da0f4.pdf,https://www.perdoes.mg.gov.br/flex/cms_downloa...,159735468355964,b'application/pdf',2020-08-13 21:38:18.903877
4,dad98a440fb8f11d055e9376e20c2058.pdf,https://www.perdoes.mg.gov.br/flex/cms_downloa...,159735468355964,b'application/pdf',2020-08-13 21:38:20.608063
...,...,...,...,...,...
844,f047326c1a298239b53a171f2eba63c8.pdf,https://www.perdoes.mg.gov.br/flex/cms_downloa...,159735468355964,b'application/pdf',2020-08-13 22:13:04.662735
845,022698c1bce0fcc7db57bea96227fa82.pdf,https://www.perdoes.mg.gov.br/flex/cms_downloa...,159735468355964,b'application/pdf',2020-08-13 22:13:08.596612
846,7e6fd230cf165aa425ddc371d4a4ef79.pdf,https://www.perdoes.mg.gov.br/flex/cms_downloa...,159735468355964,b'application/pdf',2020-08-13 22:13:10.340622
847,82ffe720490e7dfe4618068bc2ef8a4d.pdf,https://www.perdoes.mg.gov.br/flex/cms_downloa...,159735468355964,b'application/pdf',2020-08-13 22:13:12.689648


In [17]:
file_description_perdoes = identifica_atas_df(file_description_perdoes)
file_description_perdoes[[
    'ata_from_url',
    'ata_from_file_name',
    'ata_from_file_title',
    'ata_from_file_content'
]].sum()

2a1a42a986bf401016893a821ea66a10.msword: EOF marker not found
aab916a98d68b9ef946f6f2dd08cbf64.msword: EOF marker not found
5b9413d4d9319b8a29a98a20dd3f90fc.msword: EOF marker not found
6c54ac8d17766eb3a16c8faabef35434.msword: EOF marker not found
3ea7f4b61a3fd84ec448b5d2f148bb87.msword: EOF marker not found
80e229199c5f53fefabc31c111b15405.msword: EOF marker not found
21e219f68c29d183a7dc81bd4c5578f2.msword: EOF marker not found
7457a87dc49c1fe79a8d75e3599eb14d.msword: EOF marker not found
ef7aac9522b3fc6f7bc836f73c81df9b.msword: EOF marker not found
7f39c930ab2b2d3e3f9b5ce7b2ee4d22.msword: EOF marker not found
9275d07d38620a105086d4906f9f55b4.octet-stream: EOF marker not found
cf7c04521144674766af1e2e0650a6b5.msword: EOF marker not found
73786a6ad5a280db9f4c5e4ca11d3b7e.msword: EOF marker not found
3d87f9ee49328a947a036abf0a6ac490.msword: EOF marker not found
ccc509342a08160bbf2ff89d2f047b55.msword: EOF marker not found
b9b774a1be5fcfd50adb5b29af29793b.msword: EOF marker not found
66

b41db896580a151742ff1a220fe03e5a.msword: EOF marker not found
3d41a42794626a2c793d350870abbf00.octet-stream: EOF marker not found
e80ede60ea9453d911a699d1b5a75cdf.msword: EOF marker not found
8a63a22de14123b51994e4c776ed9aee.msword: EOF marker not found
c7ea0d6dd249990c25d0ab60ebac221d.msword: EOF marker not found
63726cd2d46cfb9c66ae82e6082c90e3.msword: EOF marker not found
2f53222a4132dee2a0e6d92163b0e68f.octet-stream: EOF marker not found
d9bfed752de07e7486d1fa16f9918331.msword: EOF marker not found
54b049481d9aeb1af3543efd48687f33.vnd.openxmlformats-officedocument.wordprocessingml.document: EOF marker not found
b7bdcb28fab47fff62834ced23e28d79.msword: EOF marker not found
803384d9d5b212085648b30ae12e4504.msword: EOF marker not found
b6e38af6c338848bbede4efbc37e3f0c.msword: EOF marker not found
e0af86c183ec6d1f8732f60837896dab.msword: EOF marker not found
d915bbc9a026621e452374dfaed3e848.octet-stream: EOF marker not found
41300eea72ee75bee97af2411f552a82.msword: EOF marker not found

c237855cf0dac35d4cc187a77985282e.vnd.openxmlformats-officedocument.wordprocessingml.document: EOF marker not found
aa409448d68b567577aa1d4ed531376e.msword: EOF marker not found
27285863b2f705719805f0b86e0f2de7.msword: EOF marker not found
3760eb5eeca0bff5b94a7b33f32410bb.msword: EOF marker not found
320f7c8c389783c18ba324c1d2e399e7.msword: EOF marker not found
b1628dcb4607c513ece4db7331638472.msword: EOF marker not found
f631d027483ac77b42f59416a66e0450.msword: EOF marker not found
0418f700ca3e6a6dd9a62d4e47b35f2f.msword: EOF marker not found
1d02c9dc1c74a192739f7e3536f8579c.msword: EOF marker not found
2b9d917fe3dfb1cae4c9204871755604.vnd.openxmlformats-officedocument.wordprocessingml.document: EOF marker not found
024a53130bf1d2812e1527e7186ec13c.msword: EOF marker not found
e1584d116257ad936dc430b926ef0be1.msword: EOF marker not found
630ec409b794b69cefe7a966b852e256.vnd.openxmlformats-officedocument.wordprocessingml.document: EOF marker not found
0f0385f90134b2c149ada20f7473a32c.ms

ce511638977a193de981944202b69afa.msword: EOF marker not found
8f2790d45d5b094e4083ba603828defa.msword: EOF marker not found


9e9e787a17a8fbab36325e627097ef78.vnd.openxmlformats-officedocument.wordprocessingml.document: EOF marker not found
9364f78174fc6be5719c13d4bd72c971.msword: EOF marker not found
ab04225cd53cc392cfbf2936b791d767.msword: EOF marker not found
c4bd1ef3eb0af20ce9437a1ee46412a9.vnd.openxmlformats-officedocument.wordprocessingml.document: EOF marker not found
9143eb422f72fee7d304b3b44c266099.msword: EOF marker not found
b47f58e4509c8204fd77d34dd5cb002e.msword: EOF marker not found
dfb4292e66e7aa31360179ef5a31be4b.msword: EOF marker not found
41f6a6b68baa16a7b8e8052ae96a85a5.msword: EOF marker not found
4745169d06358e3b7f0221fdfb53579b.msword: EOF marker not found
c741f69c9faf5976dadfacaae7b867ab.msword: EOF marker not found
c1e97e774b18766572805019701f4bc5.msword: EOF marker not found
8910cef63303153a7f0cf56f1c86b42f.msword: EOF marker not found
506a56db562c8a8bbbcd3af09d576df1.msword: EOF marker not found
8c1bc8117cf986b292ca35326aabf2fe.msword: EOF marker not found
ff52df0db6fe535f1a78c2948d

64128578a86fd3a78dcbfabf32b6dfa1.msword: EOF marker not found
31df46f65055dc335bf68ee40cda44a3.vnd.openxmlformats-officedocument.wordprocessingml.document: EOF marker not found
f851a78c85a50a72bafe7d1b00f5089e.msword: EOF marker not found


ad98d7bfb7a7c5b328e754a22cc31a3e.vnd.openxmlformats-officedocument.wordprocessingml.document: EOF marker not found



ata_from_url               0
ata_from_file_name         0
ata_from_file_title       22
ata_from_file_content    524
dtype: int64

In [18]:
# Seleciona aleatoriamente amostras de arquivos
filter_ = file_description_perdoes[[
    'ata_from_url',
    'ata_from_file_name',
    'ata_from_file_title',
    'ata_from_file_content'
]].sum(axis=1)

# Não foi detectada nenhuma presença da palavra ata
nao_contem_ata = file_description_perdoes.loc[filter_ == 0, 'file_name'].sample(20, random_state=44).tolist()
# Detectada alguma presença da palavra ata
contem_ata = file_description_perdoes.loc[filter_ > 0, 'file_name'].sample(20, random_state=44).tolist()
# Onde foi encontrada a palavra ata
file_description_perdoes.loc[filter_ > 0, ['file_name',
                                             'ata_from_url',
                                             'ata_from_file_name',
                                             'ata_from_file_title',
                                             'ata_from_file_content']].sample(20, random_state=44)

,file_name,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content
406,f836b4490c109632c782e86e3e5e30fd.pdf,0,0,0,1
345,96aad9ebabe0f7d597366fd655dc2869.pdf,0,0,0,1
223,841fe40a3e77d6c0adb8f9d2f9856cc9.pdf,0,0,0,1
334,4ec3cb86d29d75b4db83275c61582d61.pdf,0,0,0,1
178,80aa666e7fa17ed81ca9581683e19bbc.pdf,0,0,0,1
775,aa1c62a28c3342306e371db8dc3a760d.pdf,0,0,0,1
239,cd977882610cf2d9b2ac4b54b9ecb7da.pdf,0,0,0,1
723,5ebc65dfe9d814ab76154dbcd179a5d8.pdf,0,0,0,1
699,1b97478682210ff55689596c78a41a75.pdf,0,0,0,1
85,9f356d3cae26d61d282c9537b25aac67.msword,0,0,0,1


In [19]:
# Copia os arquivos sorteados para a pasta download
# a pasta será compactada e baixada para análise local
os.makedirs(os.path.join(root_dir, "download"), exist_ok=True)
all_files = contem_ata + nao_contem_ata
for file in all_files:
    src = os.path.join(root_dir, file)
    dst = os.path.join(root_dir, "download", file)    
    shutil.copy(src, dst)

In [20]:
! cd "../data/br.gov.mg.perdoes/159735468355964/files/" && zip -r "download/download.zip" "download" && cd -

  adding: download/ (stored 0%)
  adding: download/f836b4490c109632c782e86e3e5e30fd.pdf (deflated 3%)
  adding: download/96aad9ebabe0f7d597366fd655dc2869.pdf (deflated 5%)
  adding: download/841fe40a3e77d6c0adb8f9d2f9856cc9.pdf (deflated 4%)
  adding: download/4ec3cb86d29d75b4db83275c61582d61.pdf (deflated 27%)
  adding: download/80aa666e7fa17ed81ca9581683e19bbc.pdf (deflated 3%)
  adding: download/aa1c62a28c3342306e371db8dc3a760d.pdf (deflated 5%)
  adding: download/cd977882610cf2d9b2ac4b54b9ecb7da.pdf (deflated 4%)
  adding: download/5ebc65dfe9d814ab76154dbcd179a5d8.pdf (deflated 5%)
  adding: download/1b97478682210ff55689596c78a41a75.pdf (deflated 4%)
  adding: download/9f356d3cae26d61d282c9537b25aac67.msword (deflated 82%)
  adding: download/9c78aaae9353234b40eeab72d524e4ce.pdf (deflated 4%)
  adding: download/cb3f9545e1b0659b52b083407a15536f.pdf (deflated 6%)
  adding: download/c98cf8109a9e78476b56e62d0a019f5a.msword (deflated 69%)
  adding: download/4449cc1021a2531010f486e380ede9

In [21]:
# Verifica tipo dos arquivos que contem ata
file_description_perdoes[file_description_perdoes['file_name'].isin(contem_ata)][['file_name',
                                                                               'ata_from_url',
                                                                               'ata_from_file_name',
                                                                               'ata_from_file_title',
                                                                               'ata_from_file_content']]

,file_name,ata_from_url,ata_from_file_name,ata_from_file_title,ata_from_file_content
85,9f356d3cae26d61d282c9537b25aac67.msword,0,0,0,1
120,85ec597418eb8990a1a94ed3d8fa7984.msword,0,0,0,1
178,80aa666e7fa17ed81ca9581683e19bbc.pdf,0,0,0,1
220,f90eb7f14863caa7af5b7fba52e6d92c.pdf,0,0,0,1
223,841fe40a3e77d6c0adb8f9d2f9856cc9.pdf,0,0,0,1
239,cd977882610cf2d9b2ac4b54b9ecb7da.pdf,0,0,0,1
334,4ec3cb86d29d75b4db83275c61582d61.pdf,0,0,0,1
345,96aad9ebabe0f7d597366fd655dc2869.pdf,0,0,0,1
403,320d0d00a782703c698efd1724895d57.pdf,0,0,0,1
406,f836b4490c109632c782e86e3e5e30fd.pdf,0,0,0,1


In [22]:
all_files

['f836b4490c109632c782e86e3e5e30fd.pdf',
 '96aad9ebabe0f7d597366fd655dc2869.pdf',
 '841fe40a3e77d6c0adb8f9d2f9856cc9.pdf',
 '4ec3cb86d29d75b4db83275c61582d61.pdf',
 '80aa666e7fa17ed81ca9581683e19bbc.pdf',
 'aa1c62a28c3342306e371db8dc3a760d.pdf',
 'cd977882610cf2d9b2ac4b54b9ecb7da.pdf',
 '5ebc65dfe9d814ab76154dbcd179a5d8.pdf',
 '1b97478682210ff55689596c78a41a75.pdf',
 '9f356d3cae26d61d282c9537b25aac67.msword',
 '9c78aaae9353234b40eeab72d524e4ce.pdf',
 'cb3f9545e1b0659b52b083407a15536f.pdf',
 'c98cf8109a9e78476b56e62d0a019f5a.msword',
 '4449cc1021a2531010f486e380ede97a.msword',
 'c52338b715b43087f8b46948273229d4.msword',
 'f90eb7f14863caa7af5b7fba52e6d92c.pdf',
 '590d3b206978bc74a75b590da24ba05e.pdf',
 '320d0d00a782703c698efd1724895d57.pdf',
 '85ec597418eb8990a1a94ed3d8fa7984.msword',
 '22b19787e01a29e804c6adf4ccb9a3e4.msword',
 '9695ccfbb102452b97726e4e3288c84b.pdf',
 '86ff27401fc99636e08a18491d4bd1ee.pdf',
 '7945fbf55c30121b0bb8e851ecb647a2.pdf',
 '725eaa541c9364bda99786c9266bfa71.pdf'